In [0]:
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from sklearn.model_selection import train_test_split
from pyvi import ViTokenizer, ViPosTagger
import glob

In [0]:
os.getcwd()

'/Users/susu/Desktop/Nuoi'

## Read data from file. Split train, test set

In [0]:
import string

In [0]:
path=os.getcwd()
i=0
X_train=np.asarray([])
X_test=np.asarray([])
y_train=np.asarray([])
y_test=np.asarray([])
names=np.asarray([])

for file in glob.glob('vnexpress_top200/*.txt'):
    f= open('/'.join([path,file]),'r')
    content= f.readlines()
    content= [x.strip() for x in content]
    #content= [remove_punctuation(x) for x in content]
    #content = [ViTokenizer.tokenize(x.strip()) for x in content] 
    content = [x for x in content if len(x)>5]
    lable= [i]*len(content)
    names=np.append(names,file[17:-4])
    X_sub_train, X_sub_test, y_sub_train, y_sub_test= train_test_split(content, lable, test_size=0.2, shuffle=True, random_state=0)
    X_train= np.append(X_train, X_sub_train)
    X_test= np.append(X_test,X_sub_test)
    y_train=np.append(y_train,y_sub_train)
    y_test= np.append(y_test,y_sub_test)
    i= i+1   
    f.close()

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn import linear_model

In [0]:
logistic_model= linear_model.LogisticRegression(C=1000, solver='',multi_class='multinomial',)

## Using Logistic Regression to Classify

In [0]:

text_clf_logistic = Pipeline([('vect', CountVectorizer(ngram_range = (1,2))),('tfidf', TfidfTransformer(sublinear_tf=True)),('clf',logistic_model)])

In [0]:
text_clf_logistic = text_clf_logistic.fit(X_train, y_train)

In [0]:
y_predicted_logistic= text_clf_logistic.predict(X_test
                                               )

In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
print(classification_report(y_test, y_predicted_logistic, target_names= names))
print(accuracy_score(y_test, y_predicted_logistic))

             precision    recall  f1-score   support

    du-lich       0.84      0.84      0.84       104
  phap-luat       0.94      0.95      0.95       105
khoi-nghiep       0.89      0.95      0.92       106
 oto-xe-may       0.97      0.96      0.97       104
    thoi-su       0.81      0.80      0.80       104
       cuoi       0.99      0.94      0.96       102
   suc-khoe       0.93      0.95      0.94       101
     so-hoa       1.00      0.40      0.57         5
   the-thao       0.99      0.98      0.99       104
   gia-dinh       0.91      0.83      0.87       104
 kinh-doanh       0.92      0.86      0.89       102
   giai-tri       0.92      0.90      0.91       105
   the-gioi       0.91      0.90      0.91       105
     tam-su       0.88      1.00      0.93       105
   khoa-hoc       0.87      0.90      0.89       105

avg / total       0.91      0.91      0.91      1461

0.9110198494182067


## Using SVM to classify data

In [0]:
from sklearn import multiclass
from sklearn.linear_model import SGDClassifier
from sklearn import svm

In [0]:
from sklearn import svm
svc= svm.SVC(kernel='linear',C=100000)
model= multiclass.OneVsRestClassifier(svc)

In [0]:
from sklearn.metrics import classification_report
from sklearn.decomposition import pca
text_clf_svm=  Pipeline([('vect', CountVectorizer(ngram_range = (1,2))),('tfidf',TfidfTransformer()),('clf', model)])
text_clf_svm = text_clf_svm.fit(X_train, y_train)
y_predicted_svm= text_clf_svm.predict(X_test)
print(classification_report(y_test, y_predicted_svm,target_names= names))

             precision    recall  f1-score   support

    du-lich       0.83      0.82      0.83       104
  phap-luat       0.94      0.97      0.95       105
khoi-nghiep       0.87      0.94      0.90       106
 oto-xe-may       0.99      0.97      0.98       104
    thoi-su       0.82      0.79      0.80       104
       cuoi       0.99      0.92      0.95       102
   suc-khoe       0.92      0.95      0.94       101
     so-hoa       1.00      0.40      0.57         5
   the-thao       0.97      0.98      0.98       104
   gia-dinh       0.93      0.85      0.88       104
 kinh-doanh       0.93      0.85      0.89       102
   giai-tri       0.90      0.91      0.91       105
   the-gioi       0.91      0.91      0.91       105
     tam-su       0.91      1.00      0.95       105
   khoa-hoc       0.88      0.92      0.90       105

avg / total       0.91      0.91      0.91      1461



## Using MLP to Classify

In [0]:
from sklearn.neural_network import MLPClassifier

In [0]:
mlp= MLPClassifier(hidden_layer_sizes=(100), activation='logistic',)

In [0]:
from sklearn.metrics import classification_report
text_clf_mlp=  Pipeline([('vect', CountVectorizer(ngram_range=(2,1))),('tfidf', TfidfTransformer(norm='l2')), ('clf', mlp)])
text_clf_mlp= text_clf_mlp.fit(X_train, y_train)
y_predicted_mlp= text_clf_mlp.predict(X_test)
print(classification_report(y_test, y_predicted_mlp,target_names= names))

NameError: name 'Pipeline' is not defined

In [0]:
import pickle
filename = 'logistic_model.sav'
pickle.dump(text_clf_logistic, open(filename, 'wb'))
names

In [0]:
import pickle
filename = 'svm_model.sav'
pickle.dump(text_clf_svm, open(filename, 'wb'))

In [0]:
import pickle
filename = 'mlp_model.sav'
pickle.dump(text_clf_mlp, open(filename, 'wb'))


In [0]:
def predict(news, model):
    news= ViTokenizer.tokenize(news)
    news= [news]
    lable= model.predict(news)
    return names[int(lable)]

In [0]:
filename='svm_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))